# Attribution Visualization
This notebook loads the per-sample attribution files, decodes each contributing example back into text, and visualizes the top contributors.

In [18]:
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import tiktoken

# 1) Load all attribution pickles
attribution_dir = 'out-shakespeare-char'  # adjust if your logs live elsewhere
files = glob.glob(f"{attribution_dir}/attribution_iter*.pkl")

records = []
for fpath in files:
    with open(fpath, 'rb') as f:
        d = pickle.load(f)
    it = d['iter']
    for text, score, example in zip(d['data'], d['scores'], d['validation_example']):
        records.append({'validation':example, 'iter': it, 'text': text, 'score': score})

df = pd.DataFrame(records)
# pick the top 10 highest-scoring samples overall
top = df.sort_values('score', ascending=False).head(10)
top

,validation,iter,text,score
21433,i,0,"wer chair, sir; 'twas in\nthe Bunch of Grapes,...",0.728701
23641,i,2,me the duty which\nTo a mother's part belongs....,0.683721
21435,a,0,"ly, gentle sirs:\nIt will be pastime passing e...",0.559484
20632,",",4,"rth, to earth resign; end motion here;\nAnd th...",0.521375
22091,I,1,"oor lord, he is mew'd up.\nI would to God my h...",0.478783
21437,,0,"power you have.\n\nISABELLA:\nMy power? Alas, ...",0.477432
22081,i,1,"marry, did I but I was fain to forswear it;\n...",0.463036
21432,r,0,"es.\nWelcome, how agreed?\n\nISABELLA:\nShe'll...",0.450141
23645,,2,"chance entering\ninto some monastery; but, by ...",0.442544
23644,",",2,"is, being tender, raw and young:\nWhich elder ...",0.442199


In [12]:
top['text']

21433    y. If you are learn'd,\nBe not as common fools...
22083    es I prize\nAs the dead carcasses of unburied ...
22906     candle-holder, and look on.\nThe game was ne'...
22082    n.\nThis peace is nothing, but to rust iron, i...
21435    oor lord, he is mew'd up.\nI would to God my h...
22898    r age;\nThe parents live, whose children thou ...
22900    the kennel of thy womb hath crept\nA hell-houn...
21441    chance entering\ninto some monastery; but, by ...
23642    by her own report.\n\nESCALUS:\nSay you?\n\nLU...
22901    us 'head below his knee\nAnd tread upon his ne...
Name: text, dtype: object

In [ ]:

# 2) Decode token sequences back into text
import numpy as np
# from transformers import GPT2TokenizerFast

# --- adjust these to your setup ---
DATA_DIR = 'data/shakespeare_char'       # should contain train.bin
BLOCK_SIZE = 1024                 # same as your --block_size
# --------------------------------
meta_path= 'data/shakespeare_char/meta.pkl'
print(f"Loading meta from {meta_path}...")
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)
# TODO want to make this more general to arbitrary encoder/decoder schemes
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = np.memmap(
    f"{DATA_DIR}/train.bin",
    dtype=np.uint16,
    mode='r'
)

# map each index → text string
texts = []
for idx in top['index']:
    tokens = data[idx : idx + BLOCK_SIZE]
    # convert to int list then decode
    texts.append(decode(tokens.tolist()))
top = top.reset_index(drop=True)
top['text'] = texts

# 3) Plot and display
plt.figure(figsize=(8, 4))
plt.bar(range(len(top)), top['score'])
plt.xticks(range(len(top)), [f"#{i}" for i in top.index], rotation=45)
plt.ylabel('Attribution score')
plt.xlabel('Sample rank')
plt.title('Top 10 Contributing Samples')
plt.tight_layout()
plt.show()

# show the DataFrame with actual text
top[['iter', 'score', 'text']]